In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/MyDrive/yolov8_deepsort'

/content/drive/.shortcut-targets-by-id/1XGBg5Si1XywMeiU8_lIkogivVWtTut6b/yolov8_deepsort


In [ ]:
# !git clone 'https://github.com/computervisioneng/object-tracking-yolov8-deep-sort.git'

Cloning into 'object-tracking-yolov8-deep-sort'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 20 (delta 6), reused 4 (delta 2), pack-reused 7
Receiving objects: 100% (20/20), 5.50 KiB | 704.00 KiB/s, done.
Resolving deltas: 100% (6/6), done.


In [ ]:
!pip install tensorflow==2.11.0
!pip install scikit-image==0.19.3
!pip install filterpy==1.4.5
!pip install ultralytics==8.0.33

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 51.7 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Att

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=93c61263de9ebd151728d70604dd81dc070b61c5445f22cd903d3bb12fb13d54
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 14.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl

In [ ]:
import os
import random
import cv2
from ultralytics import YOLO

from tracker import Tracker

/content/drive/.shortcut-targets-by-id/1XGBg5Si1XywMeiU8_lIkogivVWtTut6b/yolov8_deepsort/deep_sort/deep_sort/linear_assignment_me.py:18: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  warnings.warn(


In [ ]:
%pwd

'/content/drive/.shortcut-targets-by-id/1XGBg5Si1XywMeiU8_lIkogivVWtTut6b/yolov8_deepsort'

In [ ]:


video_path = '/content/drive/MyDrive/test_videos_1/cars_hd.mp4'
video_out_path ='./output_videos/cars_output_hd_1.mp4'

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()

cap_out = cv2.VideoWriter(video_out_path, cv2.VideoWriter_fourcc(*'MP4V'), cap.get(cv2.CAP_PROP_FPS),
                          (frame.shape[1], frame.shape[0]))

model = YOLO("yolov8n.pt")

tracker = Tracker()

colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for j in range(10)]

detection_threshold = 0.5
while ret:

    results = model(frame)

    for result in results:
        detections = []
        for r in result.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = r
            x1 = int(x1)
            x2 = int(x2)
            y1 = int(y1)
            y2 = int(y2)
            class_id = int(class_id)
            if score > detection_threshold:
                detections.append([x1, y1, x2, y2, score])
                # print(detections)

        tracker.update(frame, detections)

        for track in tracker.tracks:
            bbox = track.bbox
            x1, y1, x2, y2 = bbox
            track_id = track.track_id


            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (colors[track_id % len(colors)]), 12)
            cv2.putText(frame, str(track_id),(int(x1), int(y1)),cv2.FONT_HERSHEY_SIMPLEX,5, (colors[track_id % len(colors)]), 5)

    cap_out.write(frame)
    ret, frame = cap.read()

cap.release()
cap_out.release()
cv2.destroyAllWindows()

Streaming output truncated to the last 5000 lines.
0: 384x640 22 cars, 1 bus, 1 truck, 12.6ms
Speed: 0.6ms pre-process, 12.6ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 20 cars, 1 truck, 20.4ms
Speed: 0.6ms pre-process, 20.4ms inference, 8.4ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 19 cars, 1 bus, 1 truck, 11.1ms
Speed: 0.6ms pre-process, 11.1ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 21 cars, 1 bus, 1 truck, 11.0ms
Speed: 0.6ms pre-process, 11.0ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 22 cars, 1 bus, 1 truck, 23.1ms
Speed: 0.6ms pre-process, 23.1ms inference, 4.7ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 24 cars, 1 bus, 1 truck, 11.4ms
Speed: 0.6ms pre-process, 11.4ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 26 cars, 1 bus, 2 trucks, 11.1ms
Speed: 0.6ms pre-process, 11.1ms inference, 2.2ms postproces